# Generate Output
This notebook will read in the metadata from the [Arquin Spreadsheet](https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431) and product an output for ingestion into Omeka 

In [2]:
url = 'https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431'

## import libraries

In [3]:
from IPython.display import clear_output
import pandas as pd
from pandas import Series
import numpy as np
import re
import csv
from gsheets import Sheets
from datetime import datetime

In [4]:
%run functions.ipynb

## Load the spreadsheet information

In [5]:
# if HttpAccessTokenRefreshError, close everything and start over
sheets = Sheets.from_files('~/client_secrets.json', '~/storage.json')
s = sheets.get(url)

## This is testing to figure out how to get all the data where it needs to go!

### Get the list of Identifiers by Box Number

In [6]:
# get the list of boxes containing metadata

box_list = []
box_list_sheets = s.sheets.titles()

for b in box_list_sheets:
    if 'Box' in b:
        box_list.append(int(b[4:]))

In [ ]:
box_list

In [8]:
x = return_box(box_list[0])

In [ ]:
# concepts[concepts['join_concept'].notna()]
'''strip leading and trailing space'''
 
x['title'] = x['title'].str.strip()

### Get the title translations
from Box 'unique_titles'

In [ ]:
title_translations = s.find('unique_titles').to_frame()

In [ ]:
title_translations

In [ ]:
box_with_trans = x.merge(title_translations, how='left', left_on='title', right_on='Title', indicator=True)

In [ ]:
box_with_trans[['identifier', 'media', 'title'
    ,'Spanish (from Google)' 
    ,'Translation ES', 'Portuguese(from Google)', 'Translation PT', '_merge'
]]

In [ ]:
%run arquin_metadata_subjects_review_final.ipynb

In [ ]:
box_with_trans[box_with_trans['identifier']=='A-1 02'][['identifier', 'media', 'title'
    ,'Spanish (from Google)' 
    ,'Translation ES', 'Portuguese(from Google)', 'Translation PT'
                                                        ,
]]

In [ ]:
x[x['identifier']=='A-1 02'][['title']]

In [ ]:
df[df['identifier']== 'A-1 02']

In [ ]:
box_with_trans[box_with_trans['identifier']=='A-1 02']

In [ ]:
box_with_trans_and_subjects = box_with_trans.merge(df, on=['media'])

In [ ]:
box_with_trans_and_subjects[box_with_trans_and_subjects['identifier_x']=='A-1 02']

In [85]:
%run arquin_metadata_subjects_review_final.ipynb

16221 of 16222: search_media = Arquin_069_0217.jpg


In [97]:
box_with_trans[box_with_trans['identifier']=='A-1 02'][['identifier', 'media', 'title'
    ,'Spanish (from Google)' 
    ,'Translation ES', 'Portuguese(from Google)', 'Translation PT'
                                                        ,
]]

,identifier,media,title,Spanish (from Google),Translation ES,Portuguese(from Google),Translation PT
0,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","Ostend Gate, el Ezelpoort",NaN,"OSTEND GATE, O EZELPOORT",NaN


In [88]:
x[x['identifier']=='A-1 02'][['title']]

,title
0,"Ostend Gate, The Ezelpoort"


In [98]:
df[df['identifier']== 'A-1 02']

,identifier,media,title,subject,description,creator,publisher,date,rights,language,...,QUALIFIER,TERM_TYPE,PART_OF_SPEECH,LANG_STAT,full_concept,use_concept,BoxId,Unnamed: 0,color,RGB
0,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,...,NaN,D,PN,U,NaN,water gates,Box 1,64,orange,"(100, 60, 0)"
1,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,...,NaN,D,PN,U,NaN,water gates,Box 1,64,orange,"(100, 60, 0)"
2,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,...,NaN,D,PN,U,NaN,arch bridges,Box 1,64,orange,"(100, 60, 0)"
3,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,...,waterways,D,PN,U,canals (waterways),canals (waterways),Box 1,64,orange,"(100, 60, 0)"


In [96]:
box_with_trans[box_with_trans['identifier']=='A-1 02']

,identifier,media,title,subject,description,creator,publisher,date,rights,language,...,coverage,spatial,Index,Title,Spanish (from Google),Translation ES,EmptyColumn,Portuguese(from Google),Translation PT,_merge
0,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,...,51.2093/ 3.2247,"Bruges (inhabited place), West Flanders (provi...",1,"Ostend Gate, The Ezelpoort","Ostend Gate, el Ezelpoort",NaN,NaN,"OSTEND GATE, O EZELPOORT",NaN,both


In [99]:
box_with_trans_and_subjects = box_with_trans.merge(df, on=['media'])

In [101]:
box_with_trans_and_subjects[box_with_trans_and_subjects['identifier_x']=='A-1 02']

,identifier_x,media,title_x,subject_x,description_x,creator_x,publisher_x,date_x,rights_x,language_x,...,QUALIFIER,TERM_TYPE,PART_OF_SPEECH,LANG_STAT,full_concept,use_concept,BoxId,Unnamed: 0,color,RGB
0,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,...,NaN,D,PN,U,NaN,water gates,Box 1,64,orange,"(100, 60, 0)"
1,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,...,NaN,D,PN,U,NaN,water gates,Box 1,64,orange,"(100, 60, 0)"
2,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,...,NaN,D,PN,U,NaN,arch bridges,Box 1,64,orange,"(100, 60, 0)"
3,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,...,waterways,D,PN,U,canals (waterways),canals (waterways),Box 1,64,orange,"(100, 60, 0)"
